# Problema clasificación con RNA y Keras

In [1]:
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import random

In [2]:
from numpy.random import seed
seed(1)

In [3]:
# Cargamos el conjunto de datos
os.chdir (os.getcwd())
df = pd.read_csv(r"datos/diabetes.csv", sep=",", header=None)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       768 non-null    int64  
 1   1       768 non-null    int64  
 2   2       768 non-null    int64  
 3   3       768 non-null    int64  
 4   4       768 non-null    int64  
 5   5       768 non-null    float64
 6   6       768 non-null    float64
 7   7       768 non-null    int64  
 8   8       768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [5]:
df.head()

,0,1,2,3,4,5,6,7,8
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


Significado de las columnas:

 0. Number of times pregnant
 1. Plasma glucose concentration a 2 hours in an oral glucose tolerance test
 2. Diastolic blood pressure (mm Hg)
 3. Triceps skin fold thickness (mm)
 4. 2-Hour serum insulin (mu U/ml)
 5. Body mass index (weight in kg/(height in m)^2)
 6. Diabetes pedigree function
 7. Age (years)
 8. Variable clase (Tiene diabetes)

In [6]:
# Dividimos los datos en X e y
X = df.iloc[:,0:8]
y = df.iloc[:,8]

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
                                        X,
                                        y,
                                        train_size   = 0.7,
                                        random_state = 123,
                                        shuffle      = True
                                    )

### Definimos el modelo Keras

In [8]:
# Para obtener todos el mismo resultado debemos añadir una semilla
tf.random.set_seed(1)
np.random.seed(1)
random.seed(1)
seed(1)

In [9]:
# Definiremos el modelo como una secuencia de capas.
# Usaremos el modelo secuencial de manera que podamos ir añadiendo capas hasta estar contentos con la arquitectura desarrollada.
model = Sequential()

# Partimos de un sistema con 8 variables por lo que nuestra primera capa acomodará dichas variables
# En la primera capa oculta usaremos 12 neuronas y una función de activación ReLU
# En la segunda capa oculta usaremos 8 neuronas y una función de activación ReLU
# Finalmente en la de salida una neurona y función de activación sigmoide
model.add(Dense(12, input_dim=8, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Nota: Fíjate que el total de neuronas de entrada, lo definimos en la primera capa con input_dim


### Compilamos el modelo

In [10]:
# La compilación usa (internamente) librerías numéricas muy eficientes como TensorFlow además de comprobar si tenemos GPU 
# o sólo CPU
# Es necesario definir la función de pérdida que vamos a minimizar (optimizar).  Para este caso minimizaremos 
# Binary Cross Entropy puesto que funciona bien para problemas binarios de clasificación.
# Como métrica (al ser clasificación) usaremos la precisión.
# Como optimizador, usaremos el algoritmo "adam" ya que ofrece buenos resultados en un amplio abanico de problemas y 
# además de manera rápida

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              #loss="binary_crossentropy",
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=False, # usar cuando valores de predicción sean [0,1]
                                                      label_smoothing=0.0,
                                                      axis=-1,
                                                      reduction="auto",
                                                      name="binary_crossentropy"),
              metrics=['accuracy'])



### Entrenamos el modelo

In [11]:
# seed(1)
# random.set_seed(1)
# El argumento batch_size, nos permite definir el número de filas que se considerarán, antes de que los pesos del
# modelo se reajusten dentro de cada ciclo.
model.fit(X_train, y_train, epochs=150, batch_size=10)

# En el aprendizaje automático, entrenar modelos de aprendizaje profundo con grandes conjuntos de datos puede 
# requerir mucha memoria y recursos computacionales. El uso de un tamaño de lote adecuado puede ayudar a evitar 
# problemas de memoria y a mejorar la velocidad de entrenamiento.

# Por ejemplo, si tenemos un conjunto de datos de 1000 muestras y establecemos batch_size=10, esto significa 
# que se utilizarán 10 muestras a la vez para actualizar los pesos del modelo en cada iteración del entrenamiento. 
# Por lo tanto, el proceso de entrenamiento se dividirá en 100 iteraciones, una para cada lote de 10 muestras.

# El tamaño de lote adecuado depende del conjunto de datos y del modelo. En general, un tamaño de lote más grande
# puede proporcionar un mejor rendimiento de entrenamiento, pero requiere más memoria. Por otro lado, un tamaño de
# lote más pequeño puede proporcionar una mejor generalización del modelo, pero puede requerir más iteraciones para
# converger a una solución óptima.

# En resumen, batch_size es un parámetro importante en el proceso de entrenamiento de modelos de aprendizaje profundo en Keras y su elección puede afectar el rendimiento del modelo y la velocidad de entrenamiento.

Epoch 1/150
54/54 [==============================] - 0s 1ms/step - loss: 1.7432 - accuracy: 0.5829
Epoch 2/150
54/54 [==============================] - 0s 1ms/step - loss: 0.9616 - accuracy: 0.5885
Epoch 3/150
54/54 [==============================] - 0s 1ms/step - loss: 0.8385 - accuracy: 0.6406
Epoch 4/150
54/54 [==============================] - 0s 1ms/step - loss: 0.7893 - accuracy: 0.6704
Epoch 5/150
54/54 [==============================] - 0s 1ms/step - loss: 0.7885 - accuracy: 0.6741
Epoch 6/150
54/54 [==============================] - 0s 1ms/step - loss: 0.7369 - accuracy: 0.6778
Epoch 7/150
54/54 [==============================] - 0s 1ms/step - loss: 0.7298 - accuracy: 0.6685
Epoch 8/150
54/54 [==============================] - 0s 1ms/step - loss: 0.7187 - accuracy: 0.6648
Epoch 9/150
54/54 [==============================] - 0s 1ms/step - loss: 0.6767 - accuracy: 0.6797
Epoch 10/150
54/54 [==============================] - 0s 2ms/step - loss: 0.6771 - accuracy: 0.6797
Epoch 11/

Epoch 83/150
54/54 [==============================] - 0s 1ms/step - loss: 0.5543 - accuracy: 0.7095
Epoch 84/150
54/54 [==============================] - 0s 1ms/step - loss: 0.5461 - accuracy: 0.7225
Epoch 85/150
54/54 [==============================] - 0s 1ms/step - loss: 0.5590 - accuracy: 0.7393
Epoch 86/150
54/54 [==============================] - 0s 1ms/step - loss: 0.5411 - accuracy: 0.7467
Epoch 87/150
54/54 [==============================] - 0s 962us/step - loss: 0.5238 - accuracy: 0.7449
Epoch 88/150
54/54 [==============================] - 0s 1ms/step - loss: 0.5323 - accuracy: 0.7542
Epoch 89/150
54/54 [==============================] - 0s 1ms/step - loss: 0.5354 - accuracy: 0.7523
Epoch 90/150
54/54 [==============================] - 0s 1ms/step - loss: 0.5374 - accuracy: 0.7263
Epoch 91/150
54/54 [==============================] - 0s 1ms/step - loss: 0.5332 - accuracy: 0.7412
Epoch 92/150
54/54 [==============================] - 0s 1ms/step - loss: 0.5399 - accuracy: 0.726

### Evaluamos el modelo

In [12]:
# Con la red neuronal entrenada, ahora debemos evaluar cómo ha funcionado.
_, accuracy = model.evaluate(X_train, y_train)
print('Accuracy: %.2f' % (accuracy*100))

17/17 [==============================] - 0s 1ms/step - loss: 0.4888 - accuracy: 0.7654
Accuracy: 76.54


In [13]:
# No es un mal resultado, tenemos una precisión de más del 75%

### Predicciones

In [14]:
predicciones = model.predict(X_test)

# La función sigmoide nos devueve los resultados en formato probabilidad.
# Convertimos los mismos a casos, tomando como umbral 0.5
y_pred = (predicciones > 0.5).astype(int)
y_pred

8/8 [==============================] - 0s 1ms/step


array([[1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
    

In [15]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[117,  26],
       [ 40,  48]], dtype=int64)

## Ejercicios

1.Configura la red neuronal para que trabaje con 3 capas. 
- La primera con 15 neuronas y función de activación sigmoide. 
- La segunda con 10 neuronas y función de activación sigmoide.
- La tercera con 8 neuronas y función de activación ReLU.
- Una capa de salida con 1 neurona y función de activación sigmoide.

2.Configura la red neuronal para que trabaje con las 3 mismas capas del ejemplo inicial, pero esta vez usa como función de activación de la capa de salida 'softmax'

3.En compile, cambia la configuración del optimizer, de manera que en vez de Adam, usemos esta vez SGD:
optimizer=tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.1, nesterov=False, name='SGD')

3.1.Cambia ahora la función de activación de la capa de salida a sigmoide

4.En compile, cambia la configuración del loss para usar como función de pérdida CategoricalCrossentropy.

tf.keras.losses.CategoricalCrossentropy(
    from_logits=False,
    label_smoothing=0.0,
    axis=-1,
    reduction="auto",
    name="categorical_crossentropy"),


5.Prueba diferentes configuraciones a ver si consigues mejorar el resultado inicial.